test

this should be text yes

In [ ]:
#This function breaks down sentences into digestable chunks for the program to analyze

PUNCTS = (".", "?", "!") #Given list of currently accepted punctuations

def processSentence(sentence):
    chunks = []
    nextChunk = ""

    for i in sentence:
        if i in PUNCTS:
            chunks.append(nextChunk)
            nextChunk = ""

            chunks.append(i)
        elif i == " ":
            if len(nextChunk) > 0:
                chunks.append(nextChunk)
                nextChunk = ""
        else:
            nextChunk += i
    
    return chunks

In [ ]:
import random

#These functions create scaled chunks based on the given information, and then select a random chunk, see more in the documentation

#Helper function, this confines the probability range between 0 and 1
def probabilityGradient(probs):
    total = sum(probs)
    return [i/total for i in probs]

#This function returns the index of the randomly chosen section
def selectValue(probs):
    gradient = probabilityGradient(probs)
    target = random.random()
    total = 0
    result = 0

    for bound in gradient:
        total += bound

        if target < total:
            return result
        result += 1

    #technically it would never reach this point but just return the last chunk in case it didn't pick anything prior
    return len(gradient) - 1

In [1]:
trainedMatrix = {
    #"SampleWord" : [[NextWords], [NextProbs]]
}

def train(trainingSentences):
    for sen in trainingSentences:
        procSen = processSentence(sen)

        for i in range(len(procSen)):
            chk = procSen[i]
            prevChk = procSen[i-1] if i > 0 else None


            if not chk in PUNCTS and not chk in trainedMatrix.keys():
                trainedMatrix[chk] = [[], []]
            
            if prevChk is not None:
                #targetIndex = [trainedMatrix.keys].index(prevChk)

                if chk in trainedMatrix[prevChk][0]:
                    updatedWrdList = trainedMatrix[prevChk][0]
                    updatedProbList = trainedMatrix[prevChk][1]

                    chkIndex = updatedWrdList.index(chk)
                    updatedProbList[chkIndex] += 1

                    trainedMatrix.update({prevChk : [updatedWrdList, updatedProbList]})
                else:
                    updatedWrdList = trainedMatrix[prevChk][0]
                    updatedProbList = trainedMatrix[prevChk][1]

                    updatedWrdList.append(chk)
                    updatedProbList.append(1)

                    trainedMatrix.update({prevChk : [updatedWrdList, updatedProbList]})

In [ ]:
MAX_SENT_LIM = 50 #Hard stop maximum sentence length to prevent possible infinite recursion

def generate(startWrd, matrix):
    sent = [startWrd]

    if not startWrd in matrix.keys():
        print("Starting word hasn't been learnt before")

    def generateRec(prevWrd):
        if len(sent) >= MAX_SENT_LIM:
            return False
        else:
            nxtWords = matrix[prevWrd][0]
            nextProbs = matrix[prevWrd][1]

            nxtWord = nxtWords[selectValue(nextProbs)]
            sent.append(nxtWord)

            if nxtWord in PUNCTS:
                return True
            else:
                return generateRec(nxtWord)